<a href="https://colab.research.google.com/github/alexlauforgithub/M064810011_Quant/blob/master/%E8%B2%A1%E9%87%91%E8%B3%87%E8%A8%8A%E7%88%AC%E8%9F%B2%E6%A0%B8%E5%BF%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import os
import time
import datetime

## 爬取大盤歷史資料

In [ ]:
# 日期設定
first_date = datetime.datetime.today().replace(day=1).strftime('%Y-%m-%d')
start_date = '2000-01-01'
date_list = pd.date_range(start_date, first_date, freq='MS')
date_list = date_list.strftime('%Y%m%d').tolist()

In [ ]:
i='20210805'

url = 'https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=csv&date='+ '20210701' 
res = requests.get(url)
text = res.text

if len(text) == 0:
    None

else:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        cleaned_data.append([ele.replace('",\r','').replace('"','') 
                            for ele in da.split('","')])
    
    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[1])[2:].dropna().reset_index(drop=True)
    df.columns = ['date', 'open', 'high', 'low', 'close']
    #民國年轉西元年
    for ix in range(len(df['date'])):
        df['date'].iloc[ix]=df['date'].iloc[ix].replace(df['date'].iloc[ix][0:3], str(int(df['date'].iloc[ix][0:3]) + 1911))
    # 轉成數值
    df['date'] = df['date'].str.replace('/', '').astype(int)

## 上市歷史價格資料

In [ ]:
# 日期設定
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = '2004-02-11' # 證交所最早提供的日期
date_list = pd.date_range(start=start_date ,end=today_date).strftime('%Y%m%d').tolist()

In [46]:
## [x]

i='20210805'
url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ i + '&type=ALLBUT0999'
res = requests.get(url)
text = res.text

if len(text) == 0:
    print(i +" is holiday!")
else:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 16 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])
    
    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
    df['日期'] = int(i)
    df['市場'] = 'TSE'
    df = df[['日期', '市場', '證券代號', '證券名稱', '開盤價', '最高價', '最低價', '收盤價', '成交股數', '成交金額']]
    df.columns = ['date_num','mkt', 'code', 'name', 'open', 'high', 'low', 'close', 'volume_share', 'volume_value']
    
    # 取股票代碼4位數字
    df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)

    df = df.replace('--', np.nan)
    df = df.replace(',','', regex=True)
    num_feat = ['open', 'high', 'low', 'close','volume_share', 'volume_value']
    df[num_feat] = df[num_feat].astype(float)

## 上市三大法人資料

In [64]:
# 日期設定
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = '2012-05-02' # 證交所最早提供的日期
date_list = pd.date_range(start=start_date ,end=today_date).strftime('%Y%m%d').tolist()

In [135]:
# 爬取證交所籌碼資料
i = '20120601'

url = 'https://www.twse.com.tw/fund/T86?response=csv&date='+ i + '&selectType=ALLBUT0999'
res = requests.get(url)
text = res.text
ix = int(i)

if len(text) == 0 or text == '\r\n':
    print('Ho!')


elif ix < 20141201:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 12 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])

    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
    df.columns = ['code', 'name', 'fi_buy', 'fi_sell', 'fi_diff', 'sitc_buy', 'sitc_sell', 'sitc_diff', 'dl_diff', 'dl_buy', 'dl_sell', 'ins_inv_diff']
    df['date_num'] = ix

    df = df.replace('--', np.nan)
    df = df.replace(',','', regex=True)
    num_feat = ['fi_buy', 'fi_sell', 'fi_diff', 'sitc_buy', 'sitc_sell', 'sitc_diff', 'dl_buy', 'dl_sell', 'dl_diff', 'ins_inv_diff']
    df[num_feat] = df[num_feat].astype(float)

    df = df[['date_num','code', 'name', 'fi_buy', 'fi_sell', 'fi_diff', 'sitc_buy', 'sitc_sell', 'sitc_diff', 'dl_buy', 'dl_sell', 'dl_diff', 'ins_inv_diff']]
    df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)


elif ix < 20171218 and ix >= 20141201 :
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 16 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])

    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
    df.columns = [
        'code', 'name', 
        'fi_buy', 'fi_sell', 'fi_diff', 'sitc_buy', 'sitc_sell', 'sitc_diff', 
        'dl_diff', 'dl_self_buy', 'dl_self_sell', 'dl_self_diff', 'dl_hedge_buy', 'dl_hedge_sell', 'dl_hedge_diff',
        'ins_inv_diff'
    ]

    df = df.replace('--', np.nan)
    df = df.replace(',','', regex=True)

    num_feat = [
        'fi_buy', 'fi_sell', 'fi_diff', 'sitc_buy', 'sitc_sell', 'sitc_diff', 
        'dl_diff', 'dl_self_buy', 'dl_self_sell', 'dl_self_diff', 'dl_hedge_buy', 'dl_hedge_sell', 'dl_hedge_diff',
        'ins_inv_diff'    
    ]
    df[num_feat] = df[num_feat].astype(float)

    
    df['date_num'] = ix
    df['dl_buy'] = df['dl_self_buy']+df['dl_hedge_buy']
    df['dl_sell'] = df['dl_self_sell']+df['dl_hedge_sell']

    df = df[['date_num','code', 'name', 'fi_buy', 'fi_sell', 'fi_diff', 'sitc_buy', 'sitc_sell', 'sitc_diff', 'dl_buy', 'dl_sell', 'dl_diff', 'ins_inv_diff']]
    df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)


elif ix >= 20171218:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 19 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])

    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
    df.columns = [
      'code',
      'name',
      'fi_buy','fi_sell','fi_diff','fi_dl_buy','fi_dl_sell','fi_dl_diff',
      'sitc_buy','sitc_sell','sitc_diff',
      'dl_diff','dl_self_buy','dl_self_sell','dl_self_diff','dl_hedge_buy','dl_hedge_sell','dl_hedge_diff',
      'ins_inv_diff'
    ]


    df = df.replace('--', np.nan)
    df = df.replace(',','', regex=True)

    num_feat = [
      'fi_buy','fi_sell','fi_diff','fi_dl_buy','fi_dl_sell','fi_dl_diff',
      'sitc_buy','sitc_sell','sitc_diff',
      'dl_diff','dl_self_buy','dl_self_sell','dl_self_diff','dl_hedge_buy','dl_hedge_sell','dl_hedge_diff',
      'ins_inv_diff'
    ]
    df[num_feat] = df[num_feat].astype(float)

    df['date_num'] = ix
    df['dl_buy'] = df['dl_self_buy']+df['dl_hedge_buy']
    df['dl_sell'] = df['dl_self_sell']+df['dl_hedge_sell']

    df['fi_buy'] = df['fi_buy']+df['fi_dl_buy']
    df['fi_sell'] = df['fi_sell']+df['fi_dl_sell']
    df['fi_diff'] = df['fi_diff']+df['fi_dl_diff']

    df['date_num'] = ix
    df = df[['date_num','code', 'name', 'fi_buy', 'fi_sell', 'fi_diff', 'sitc_buy', 'sitc_sell', 'sitc_diff', 'dl_buy', 'dl_sell', 'dl_diff', 'ins_inv_diff']]
    df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)

In [136]:
df

,date_num,code,name,fi_buy,fi_sell,fi_diff,sitc_buy,sitc_sell,sitc_diff,dl_buy,dl_sell,dl_diff,ins_inv_diff
0,20120601,2890,永豐金,19988000.0,11749372.0,8238628.0,0.0,645000.0,-645000.0,1103000.0,2258000.0,-1155000.0,6438628.0
1,20120601,2474,可成,4449240.0,2185988.0,2263252.0,485000.0,0.0,485000.0,423000.0,489000.0,-66000.0,2682252.0
2,20120601,2610,華航,5119000.0,5716107.0,-597107.0,3865000.0,0.0,3865000.0,216000.0,831000.0,-615000.0,2652893.0
3,20120601,2823,中壽,5789000.0,2267990.0,3521010.0,0.0,0.0,0.0,459000.0,1492000.0,-1033000.0,2488010.0
4,20120601,5871,中租-KY,1204000.0,109826.0,1094174.0,743000.0,0.0,743000.0,120000.0,170000.0,-50000.0,1787174.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,20120601,2317,鴻海,9729000.0,17035608.0,-7306608.0,1433000.0,328000.0,1105000.0,712000.0,564000.0,148000.0,-6053608.0
738,20120601,2409,友達,2585000.0,6831338.0,-4246338.0,85000.0,1170000.0,-1085000.0,408000.0,1188000.0,-780000.0,-6111338.0
739,20120601,2883,開發金,18195000.0,27145411.0,-8950411.0,596000.0,2000.0,594000.0,568000.0,1159000.0,-591000.0,-8947411.0
740,20120601,2303,聯電,8095000.0,28439155.0,-20344155.0,61000.0,1000.0,60000.0,698000.0,307000.0,391000.0,-19893155.0


In [63]:
i = '20140102'

url = 'https://www.twse.com.tw/fund/T86?response=csv&date='+ i + '&selectType=ALLBUT0999'
res = requests.get(url)
text = res.text
ix = int(i) 
cleaned_data = []
for da in text.split('\n'):
    if len(da.split('","')) == 12 and da.split('","')[0][0] != '=':
        cleaned_data.append([ele.replace('",\r','').replace('"','') 
                              for ele in da.split('","')])

  # 輸出成表格並呈現在excel上
df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
df.head()

,證券代號,證券名稱,外資買進股數,外資賣出股數,外資買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數,自營商賣出股數,三大法人買賣超股數
1,2883,開發金,"5,466,000","1,766,000","3,700,000",0,"34,000","-34,000","6,553,000","8,809,000","2,256,000","10,219,000"
2,3698,隆達,"6,885,000","1,239,000","5,646,000","2,194,000","14,000","2,180,000","1,266,000","2,255,000","989,000","9,092,000"
3,2317,鴻海,"13,441,091","4,732,000","8,709,091","96,000","596,000","-500,000","-122,000","903,000","1,025,000","8,087,091"
4,2449,京元電子,"5,040,000","424,000","4,616,000","900,000","54,000","846,000","1,697,000","2,611,000","914,000","7,159,000"
5,2886,兆豐金,"5,806,000","2,280,000","3,526,000","1,500,000","167,000","1,333,000","-854,000","322,000","1,176,000","4,005,000"


In [57]:
i = '20150806'

url = 'https://www.twse.com.tw/fund/T86?response=csv&date='+ i + '&selectType=ALLBUT0999'
res = requests.get(url)
text = res.text
ix = int(i) 
cleaned_data = []
for da in text.split('\n'):
    if len(da.split('","')) == 16 and da.split('","')[0][0] != '=':
        cleaned_data.append([ele.replace('",\r','').replace('"','') 
                              for ele in da.split('","')])

# 輸出成表格並呈現在excel上
df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
df.head()

,證券代號,證券名稱,外資買進股數,外資賣出股數,外資買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
1,2412,中華電,"12,736,000","3,851,191","8,884,809","557,000","184,000","373,000","77,000","6,000","13,000","-7,000","121,000","37,000","84,000","9,334,809"
2,2884,玉山金,"13,460,000","7,656,680","5,803,320","809,000",0,"809,000","-718,000",0,"60,000","-60,000","364,000","1,022,000","-658,000","5,894,320"
3,2888,新光金,"10,601,000","5,431,000","5,170,000","45,000",0,"45,000","400,000","726,000","23,000","703,000","11,000","314,000","-303,000","5,615,000"
4,2354,鴻準,"7,580,000","3,035,100","4,544,900","251,000","295,000","-44,000","-280,000","53,000","128,000","-75,000","277,000","482,000","-205,000","4,220,900"
5,1314,中石化,"4,560,000","1,182,000","3,378,000",0,0,0,"-199,000","102,000","29,000","73,000","38,000","310,000","-272,000","3,179,000"


In [ ]:
## https://isin.twse.com.tw/isin/C_public.jsp?strMode=2

## https://isin.twse.com.tw/isin/C_public.jsp?strMode=4
## 產業對照



In [54]:
i = '20200804'

url = 'https://www.twse.com.tw/fund/T86?response=csv&date='+ i + '&selectType=ALLBUT0999'
res = requests.get(url)
text = res.text
cleaned_data = []
for da in text.split('\n'):
    if len(da.split('","')) == 19 and da.split('","')[0][0] != '=':
        cleaned_data.append([ele.replace('",\r','').replace('"','') 
                              for ele in da.split('","')])

# 輸出成表格並呈現在excel上
df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
df.head()

,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
1,3481,群創,"62,331,081","12,384,897","49,946,184",0,0,0,0,0,0,"3,506,000","638,000","422,000","216,000","3,552,000","262,000","3,290,000","53,452,184"
2,2409,友達,"51,116,868","15,936,967","35,179,901",0,0,0,0,0,0,"4,271,000","839,000",0,"839,000","3,692,000","260,000","3,432,000","39,450,901"
3,6116,彩晶,"27,404,000","2,843,000","24,561,000",0,0,0,0,0,0,"2,798,000","556,000","426,000","130,000","2,695,000","27,000","2,668,000","27,359,000"
4,3231,緯創,"14,674,665","5,886,488","8,788,177",0,0,0,"1,245,037",0,"1,245,037","975,000","369,000","83,000","286,000","879,000","190,000","689,000","11,008,214"
5,2330,台積電,"29,172,796","18,638,893","10,533,903",0,0,0,"156,000","90,000","66,000","-175,000","65,000","171,000","-106,000","174,000","243,000","-69,000","10,424,903"


## 上市指數

In [ ]:
url='https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20210806&type=IND'

In [141]:
i='20210805'
url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+i+'&type=IND'
res = requests.get(url)
text = res.text

if len(text) == 0:
    print(i +" is holiday!")
else:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 6 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])

In [144]:
df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]

In [145]:
df

,指數,收盤指數,漲跌(+/-),漲跌點數,漲跌百分比(%),特殊處理註記
1,寶島股價指數,"20,121.94",-,7.03,-0.03,
2,發行量加權股價指數,"17,603.12",-,20.77,-0.12,
3,臺灣公司治理100指數,"10,118.19",+,4.55,0.04,
4,臺灣50指數,"13,803.43",+,3.44,0.02,
5,臺灣50權重上限30%指數,"13,061.55",+,4.41,0.03,
...,...,...,...,...,...,...
176,特選台股領航報酬指數,"8,437.76",-,3.81,-0.05,
177,特選Smart多因子報酬指數,"7,171.16",-,14.13,-0.20,
178,特選ESG高股息報酬指數,"4,909.80",+,49.83,1.03,
179,特選上櫃ESG成長報酬指數,"5,355.51",+,170.68,3.29,


## 上櫃歷史價格資料

In [ ]:
import pandas as pd
from io import StringIO

In [112]:
url = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d=107/02/09&s=0,asc,0'
res = requests.get(url)
lines = res.text.replace('\r', '').split('\n')

df = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
df.columns = list(map(lambda l: l.replace(' ',''), lines[2].split(',')))


df['日期'] = int(i)
df['市場'] = 'OTC'

df = df[['日期','市場','代號','名稱','開盤','最高','最低','收盤','成交股數','成交金額(元)']]
df.columns = ['date_num','mkt', 'code', 'name', 'open', 'high', 'low', 'close', 'volume_share', 'volume_value']

# 取股票代碼4位數字
df = df[df['code'].apply(lambda x: len(x) ==4)].reset_index(drop=True)
df = df[df['code'].str.contains('[0-9]{4}')]

df = df.replace('---', np.nan)
df = df.replace(' ---', np.nan)

df = df.replace(',','', regex=True)
num_feat = ['open', 'high', 'low', 'close','volume_share', 'volume_value']
df[num_feat] = df[num_feat].astype(float)


df.head()

,date_num,mkt,code,name,open,high,low,close,volume_share,volume_value
0,20140102,OTC,1258,其祥-KY,25.0,25.9,25.0,25.75,48192.0,1230092.0
1,20140102,OTC,1259,安心,85.1,85.1,82.5,85.00,7166.0,606941.0
2,20140102,OTC,1264,德麥,224.0,224.5,223.0,224.50,59424.0,13294530.0
3,20140102,OTC,1268,漢來美食,138.0,139.0,137.0,139.00,14172.0,1951408.0
4,20140102,OTC,1333,恩得利,NaN,NaN,NaN,NaN,10.0,30.0


## 上櫃三大法人價格資料

In [133]:

## 96/11/1
url = 'https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_download.php?l=zh-tw&se=EW&t=D&d=103/11/28&s=0,asc,0'

## 2014/12/1開始

url = 'https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&o=csv&se=EW&t=D&d=103/12/01&s=0,asc'
res = requests.get(url)
lines = res.text.replace('\r', '').split('\n')


In [122]:
df = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
df.columns = list(map(lambda l: l.replace(' ',''), lines[1].split(',')))
df

,代號,名稱,外資及陸資(不含外資自營商)-買進股數,外資及陸資(不含外資自營商)-賣出股數,外資及陸資(不含外資自營商)-買賣超股數,外資自營商-買進股數,外資自營商-賣出股數,外資自營商-買賣超股數,外資及陸資-買進股數,外資及陸資-賣出股數,外資及陸資-買賣超股數,投信-買進股數,投信-賣出股數,投信-買賣超股數,自營商(自行買賣)-買進股數,自營商(自行買賣)-賣出股數,自營商(自行買賣)-買賣超股數,自營商(避險)-買進股數,自營商(避險)-賣出股數,自營商(避險)-買賣超股數,自營商-買進股數,自營商-賣出股數,自營商-買賣超股數,三大法人買賣超股數合計
0,00687B,國泰20年美債,0,"2,000","-2,000",0,0,0,0,"2,000","-2,000",0,0,0,0,0,0,"50,000","51,000","-1,000","50,000","51,000","-1,000","-3,000"
1,00694B,富邦美債1-3,0,"5,000","-5,000",0,0,0,0,"5,000","-5,000",0,0,0,0,0,0,"100,000","114,000","-14,000","100,000","114,000","-14,000","-19,000"
2,00695B,富邦美債7-10,0,"1,000","-1,000",0,0,0,0,"1,000","-1,000",0,0,0,0,0,0,"100,000","136,000","-36,000","100,000","136,000","-36,000","-37,000"
3,00696B,富邦美債20年,"1,000","12,000","-11,000",0,0,0,"1,000","12,000","-11,000",0,0,0,0,0,0,"100,000","152,000","-52,000","100,000","152,000","-52,000","-63,000"
4,00697B,元大美債7-10,"5,000","2,000","3,000",0,0,0,"5,000","2,000","3,000",0,0,0,0,0,0,"140,000","146,000","-6,000","140,000","146,000","-6,000","-3,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,020021,統一�琤肮鴔滆,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"473,000","141,000","332,000","473,000","141,000","332,000","332,000"
695,020023,元大櫃買半導體N,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"90,000","163,000","-73,000","90,000","163,000","-73,000","-73,000"
696,020025,統一亞洲半導體N,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"39,000","4,381,000","-4,342,000","39,000","4,381,000","-4,342,000","-4,342,000"
697,020026,兆豐上櫃ESG電菁N,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"22,000","20,000","2,000","22,000","20,000","2,000","2,000"
